In [ ]:
# Hello

In [ ]:
import pandas as pd
import os, sys
import datetime
import json

In [ ]:
# files = [file_path for file_path in os.listdir('CSVs') if file_path.endswith('.csv')]

In [ ]:
# df = pd.read_csv(os.path.join('CSVs', files[0]))

# for csv_path in files[1:]:
#     df_to_add = pd.read_csv(os.path.join('CSVs', csv_path))
#     df = pd.concat([df,df_to_add])
    
    
# df.reset_index(drop=True, inplace=True)

In [ ]:
COLs = ['country','location','category', 'blurb', 'slug', 'backers_count', 
       'created_at','goal','launched_at','deadline','converted_pledged_amount',
        'state','state_changed_at', 'currency','spotlight',
        'staff_pick', 'static_usd_rate','usd_type']
    

def wrangle(CSV_PATH):
    df = pd.read_csv(CSV_PATH, usecols= COLs)
    df.dropna(inplace=True)

    #Reorder columns
    df = df[['country','location','category', 'blurb', 'slug', 'backers_count', 
        'created_at','launched_at','deadline','converted_pledged_amount','goal',
        'state','state_changed_at', 'currency','spotlight',
        'staff_pick', 'static_usd_rate','usd_type']]
    
    # Dropping rows where state is = to 'live' or 'canceled'
    df = df[df['state'].isin(['successful','failed'])]
    
    # Convert str to dicts (category and location columns)
    convert_to_dicts = lambda x: json.loads(x)
    df['category'] = df['category'].apply(convert_to_dicts)
    df['location'] = df['location'].apply(convert_to_dicts)
    
    # Exctract name from dicts (category and location columns)
    extract_name = lambda x: x['name']
    df['category'] = df['category'].apply(extract_name)
    df['location'] = df['location'].apply(extract_name)
    
    # Timestamps to Dates
    convert = lambda x: datetime.datetime.fromtimestamp(x)
    df['created_at'] = df['created_at'].apply(convert)
    df['launched_at'] = df['launched_at'].apply(convert)
    df['deadline'] = df['deadline'].apply(convert)
    df['state_changed_at'] = df['state_changed_at'].apply(convert)
    
    # Rename
    df.columns = ['country','location','category', 'blurb', 'slug', 'n_backers', 
       'created_at','launch_date','deadline','pledged_$','goal',
        'state','state_changed_at', 'currency','spotlight',
          'staff_pick', 'usd_rate','usd_type']
    

    
    return df

In [ ]:
df = wrangle('Kick(2021-2022).csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208215 entries, 0 to 223288
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   country           208215 non-null  object        
 1   location          208215 non-null  object        
 2   category          208215 non-null  object        
 3   blurb             208215 non-null  object        
 4   slug              208215 non-null  object        
 5   n_backers         208215 non-null  int64         
 6   created_at        208215 non-null  datetime64[ns]
 7   launch_date       208215 non-null  datetime64[ns]
 8   deadline          208215 non-null  datetime64[ns]
 9   pledged_$         208215 non-null  int64         
 10  goal              208215 non-null  float64       
 11  state             208215 non-null  object        
 12  state_changed_at  208215 non-null  datetime64[ns]
 13  currency          208215 non-null  object        
 14  spot

In [ ]:
df.head()

,country,location,category,blurb,slug,n_backers,created_at,launch_date,deadline,pledged_$,goal,state,state_changed_at,currency,spotlight,staff_pick,usd_rate,usd_type
0,US,Burlington,Glass,I make these adorable planters myself. Custo...,i-would-like-to-produce-and-sell-my-lightbulb-...,2,2014-08-14 17:03:06,2014-08-20 14:16:23,2014-09-19 14:16:23,31,2000.0,failed,2014-09-19 14:16:23,USD,False,False,1.000000,domestic
1,US,Vineyard,Events,"The Utah Valley Food Fest is a sampling event,...",utah-valley-food-fest-2019,188,2019-07-17 22:31:20,2019-07-31 21:55:28,2019-08-17 02:01:00,3367,3000.0,successful,2019-08-17 02:01:00,USD,True,False,1.000000,domestic
2,BE,Antwerp,Comedy,Newly found cabaret project Bunny Burlesk Even...,bbe-cabaret-burlesque-show,12,2018-10-01 04:32:40,2018-10-02 05:02:05,2018-12-01 05:02:05,380,300.0,successful,2018-12-01 05:02:05,EUR,True,False,1.159643,international
3,US,Oxnard,Children's Books,A children's book about a bear that discovers ...,you-are-loved-a-bear-odd-fruit-origin-story,89,2020-12-13 19:21:05,2020-12-29 22:00:16,2021-01-19 22:30:00,4839,4500.0,successful,2021-01-19 22:30:00,USD,True,True,1.000000,domestic
4,US,New York,Musical,What happens when the queen of darkness decide...,wish-a-mystical-magical-murderful-musical,30,2015-06-28 10:42:17,2015-07-02 17:49:19,2015-08-02 19:00:00,10703,10000.0,successful,2015-08-02 19:00:16,USD,True,False,1.000000,international


In [ ]:
df['launch_week'] = [i.week for i in df['launch_date']]

In [ ]:
test = df['deadline'][1] - df['launch_date'][1]

In [ ]:
test.days

16

In [ ]:
df.nunique()

country                 25
location             10834
category               161
blurb               130836
slug                131581
n_backers             3114
created_at          131535
launch_date         131489
deadline            125114
pledged_$            29267
goal                  4541
state                    2
state_changed_at    125202
currency                15
spotlight                2
staff_pick               2
usd_rate             13715
usd_type                 2
dtype: int64

In [ ]:
datetime.datetime.fromtimestamp(df['created_at'][1])

datetime.datetime(2018, 5, 29, 9, 56, 42)

In [ ]:
df['created_at'][0].to_pydatetime()

AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'

In [ ]:
d_test = pd.read_csv('CSVs/2022 (9).csv')

In [ ]:
convert = lambda x: datetime.datetime.fromtimestamp(x)
df['created_at'] = df['created_at'].apply(convert)
# d_test['created_at'] = d_test['created_at'].apply(convert)
# d_test['launched_at'] = d_test['launched_at'].apply(convert)

In [ ]:
df['created_at'] 

0        2014-08-14 17:03:06
1        2019-07-17 22:31:20
2        2018-10-01 04:32:40
3        2020-12-13 19:21:05
4        2015-06-28 10:42:17
                 ...        
223284   2015-07-28 19:00:33
223285   2015-07-01 22:01:36
223286   2015-07-15 12:23:38
223287   2015-07-02 07:11:45
223288   2017-07-26 08:49:18
Name: created_at, Length: 222754, dtype: datetime64[ns]

In [ ]:
df['last_update_published_at'].value_counts()

1.592374e+09    1
Name: last_update_published_at, dtype: int64

In [ ]:
(d_test['deadline'][1] - d_test['created_at'][2])

Timedelta('216 days 11:17:00')

In [1]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving Kick(2021-2022)_wrangled.csv to Kick(2021-2022)_wrangled (2).csv


In [2]:
import pandas as pd

In [3]:
DF = pd.read_csv('Kick(2021-2022)_wrangled.csv').drop(columns=['Unnamed: 0'])

In [4]:
DF.head()

,country,city_name,category,description,project_name,goal,state,currency,launch_month_number_of_the_year,campaign_duration_in_days
0,US,Burlington,Glass,I make these adorable planters myself. Custo...,i would like to produce and sell my lightbulb ...,2000.0,failed,USD,8,30
1,US,Vineyard,Events,"The Utah Valley Food Fest is a sampling event,...",utah valley food fest 2019,3000.0,successful,USD,7,16
2,BE,Antwerp,Comedy,Newly found cabaret project Bunny Burlesk Even...,bbe cabaret burlesque show,300.0,successful,EUR,10,60
3,US,Oxnard,Children's Books,A children's book about a bear that discovers ...,you are loved a bear odd fruit origin story,4500.0,successful,USD,12,21
4,US,New York,Musical,What happens when the queen of darkness decide...,wish a mystical magical murderful musical,10000.0,successful,USD,7,31


In [5]:
DF['description'] = [len(x) for x in DF['description']]
DF['project_name'] = [len(x) for x in DF['project_name']]

In [6]:
DF.head()

,country,city_name,category,description,project_name,goal,state,currency,launch_month_number_of_the_year,campaign_duration_in_days
0,US,Burlington,Glass,96,50,2000.0,failed,USD,8,30
1,US,Vineyard,Events,133,26,3000.0,successful,USD,7,16
2,BE,Antwerp,Comedy,134,26,300.0,successful,EUR,10,60
3,US,Oxnard,Children's Books,133,43,4500.0,successful,USD,12,21
4,US,New York,Musical,92,41,10000.0,successful,USD,7,31


In [7]:
DF['state'] = [x.replace('failed', '0') for x in DF['state']]

In [8]:
DF['state'] = [x.replace('successful', '1') for x in DF['state']]

In [9]:
DF['state'].astype(int)

0         0
1         1
2         1
3         1
4         1
         ..
208210    0
208211    0
208212    0
208213    0
208214    1
Name: state, Length: 208215, dtype: int64

In [10]:
DF.head()

,country,city_name,category,description,project_name,goal,state,currency,launch_month_number_of_the_year,campaign_duration_in_days
0,US,Burlington,Glass,96,50,2000.0,0,USD,8,30
1,US,Vineyard,Events,133,26,3000.0,1,USD,7,16
2,BE,Antwerp,Comedy,134,26,300.0,1,EUR,10,60
3,US,Oxnard,Children's Books,133,43,4500.0,1,USD,12,21
4,US,New York,Musical,92,41,10000.0,1,USD,7,31


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
# DF = pd.read_csv('Kick(2021-2022)_wrangled.csv').drop(columns=['Unnamed: 0'])

In [12]:
vectorizer = CountVectorizer()
test = vectorizer.fit_transform(DF['country'])
DF['country'] = test.todense()

In [ ]:
vectorizer2 = CountVectorizer()
test2 = vectorizer2.fit_transform(DF['city_name'])
DF['city_name'] = test2.todense()

In [ ]:
vectorizer3 = CountVectorizer()
test3 = vectorizer3.fit_transform(DF['category'])
DF['category'] = test3.todense()

In [ ]:
DF.head()

In [47]:
DF['country'].nunique()

2

In [46]:
DF.dtypes

country                              int64
city_name                           object
category                            object
description                         object
project_name                        object
goal                               float64
state                               object
currency                            object
launch_month_number_of_the_year      int64
campaign_duration_in_days            int64
dtype: object

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [47]:
X = DF.drop(columns='state')
y = DF['state']

In [48]:
X_train, X_test = train_test_split(X, test_size=.25, train_size=.75, random_state=42)
y_train, y_test = train_test_split(y, test_size=.25, train_size=.75, random_state=42)

In [49]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(156161, 9)
(156161,)
(52054, 9)
(52054,)


In [50]:
def make_model():
  model = Sequential()
  model.add(Dense(30, activation='relu', input_shape=(156161, 9)))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy',
                optimizer='adam', 
                metrics=['accuracy'])
  return model

In [51]:
model = make_model()

In [52]:
import tensorflow as tf
import numpy as np
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [63]:
tf.convert_to_tensor(X_train)

ValueError: ignored

In [54]:
model.fit(x=X_train,
          y=y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          callbacks=['accuracy'])

ValueError: ignored